In [1]:
import numpy as np
import plotly.graph_objects as go
from qctrlvisualizer import display_bloch_sphere_from_bloch_vectors
from scipy.linalg import expm
from scipy.optimize import least_squares

from qubex.analysis.fitting import fit_rotation
from qubex.analysis.visualization import plot_bloch_vectors

In [2]:
# Generators of SO(3)

G_x = np.array(
    [
        [0, 0, 0],
        [0, 0, -1],
        [0, 1, 0],
    ]
)

G_y = np.array(
    [
        [0, 0, 1],
        [0, 0, 0],
        [-1, 0, 0],
    ]
)

G_z = np.array(
    [
        [0, -1, 0],
        [1, 0, 0],
        [0, 0, 0],
    ]
)


def rotation_matrix(
    t: float,
    Omega: float,
    n: tuple[float, float, float],
    use_expm: bool = False,
) -> np.ndarray:
    G = n[0] * G_x + n[1] * G_y + n[2] * G_z
    if use_expm:
        R = expm(Omega * t * G)
    else:
        # Rodrigues' rotation formula
        R = np.eye(3) + np.sin(Omega * t) * G + (1 - np.cos(Omega * t)) * G @ G
    return R

In [3]:
# # Parameters for the rotation
# t = 1.0
# Omega = 1.0
# n_x, n_y, n_z = 1.0, 0.0, 0.0

# # Time with %timeit for use_expm=False
# print("Time with Rodrigues' formula:")
# %timeit rotation_matrix(t, Omega, n_x, n_y, n_z, use_expm=False)

# # Time with %timeit for use_expm=True
# print("Time with expm:")
# %timeit rotation_matrix(t, Omega, n_x, n_y, n_z, use_expm=True)

In [4]:
R_x = lambda t, Omega: rotation_matrix(t, Omega, (1, 0, 0))
R_y = lambda t, Omega: rotation_matrix(t, Omega, (0, 1, 0))
R_z = lambda t, Omega: rotation_matrix(t, Omega, (0, 0, 1))

In [ ]:
Omega = 2 * np.pi
r_0 = np.array([1, 1, 1]) / np.sqrt(3)
times = np.linspace(0, 1, 50)
r_x = np.array([R_x(t, Omega) @ r_0 for t in times])
plot_bloch_vectors(times, r_x)
display_bloch_sphere_from_bloch_vectors(r_x)

In [ ]:
Omega = 4 * np.pi
r_0 = np.array([1, 1, 1]) / np.sqrt(3)
times = np.linspace(0, 1, 50)
r_y = np.array([R_y(t, Omega) @ r_0 for t in times])
plot_bloch_vectors(times, r_y)
display_bloch_sphere_from_bloch_vectors(r_y)

In [ ]:
Omega = 6 * np.pi
r_0 = np.array([1, 1, 1]) / np.sqrt(3)
times = np.linspace(0, 1, 50)
r_z = np.array([R_z(t, Omega) @ r_0 for t in times])
plot_bloch_vectors(times, r_z)
display_bloch_sphere_from_bloch_vectors(r_z)

In [8]:
def create_data(Omega, r_0, n, times, noise, delta):
    r_t = np.array([R_z(t, delta) @ rotation_matrix(t, Omega, n) @ r_0 for t in times])
    return r_t + np.random.normal(0, noise, r_t.shape)


Omega = 4 * np.pi
r_0 = np.array([0, 0, 1])
n = np.array([1.0, -0.5, 0.1])
n = n / np.linalg.norm(n)
times = np.linspace(0, 1, 50)

In [ ]:
noise = 0.05
delta = 0.0

data = create_data(Omega, r_0, n, times, noise, delta)

plot_bloch_vectors(times, data)
display_bloch_sphere_from_bloch_vectors(data)

In [ ]:
def simulate_rotation(times, Omega, theta, phi):
    n_x = np.sin(theta) * np.cos(phi)
    n_y = np.sin(theta) * np.sin(phi)
    n_z = np.cos(theta)
    return np.array(
        [
            rotation_matrix(t, Omega, (n_x, n_y, n_z)) @ r_0
            for t in times
        ]
    ).astype(float)


def residual(params, times, data):
    Omega, theta, phi = params
    return (simulate_rotation(times, Omega, theta, phi) - data).flatten()


initial_guess = [4 * np.pi, 0, 0]
result = least_squares(residual, initial_guess, args=(times, data))
result.x

In [ ]:
fit = simulate_rotation(times, *result.x)

plot_bloch_vectors(times, data, title="State evolution : data")
plot_bloch_vectors(times, fit, title="State evolution : fit")

In [ ]:
fig = go.Figure()

# data
fig.add_trace(
    go.Scatter3d(
        name="data",
        x=data[:, 0],
        y=data[:, 1],
        z=data[:, 2],
        mode="markers",
        marker=dict(size=3),
        hoverinfo="skip",
    )
)

# fit
fig.add_trace(
    go.Scatter3d(
        name="fit",
        x=fit[:, 0],
        y=fit[:, 1],
        z=fit[:, 2],
        mode="lines",
        line=dict(width=4),
        hoverinfo="skip",
    )
)

# sphere
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)
r = 1
x = r * np.sin(theta) * np.cos(phi)
y = r * np.sin(theta) * np.sin(phi)
z = r * np.cos(theta)
fig.add_trace(
    go.Surface(
        x=x,
        y=y,
        z=z,
        opacity=0.05,
        showscale=False,
        colorscale="gray",
        hoverinfo="skip",
    )
)

# layout
fig.update_layout(
    scene=dict(
        xaxis=dict(title="〈X〉", visible=True),
        yaxis=dict(title="〈Y〉", visible=True),
        zaxis=dict(title="〈Z〉", visible=True),
        aspectmode="cube",
    ),
    width=400,
    height=400,
    margin=dict(l=0, r=0, b=0, t=0),
    showlegend=False,
)

fig.show()

In [ ]:
fit_rotation(times, data, offresonant=False)
display_bloch_sphere_from_bloch_vectors(data)